# **STEP 1:** Dataset Organization

In [1]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import os
import os.path
import shutil
import zipfile
from zipfile import ZipFile
from glob import glob

### Organize Dataset of LINAC and Gamma Knife patients

### 1.1. Extract files from Zipped folder

In [29]:
# Extract folders from each pacient from file Paciente_N.zip

file = zipfile.ZipFile("/notebooks/disk2/RT_Dataset_2/00_Original_Dataset/Patient_83.zip",'r')
file.extractall("/notebooks/disk2/RT_Dataset_2/01_1_Unzipped_Dataset/Patient_83/")

root_path = "/notebooks/disk2/RT_Dataset_2/01_1_Unzipped_Dataset/Patient_83"

In [30]:
parts = root_path.split("/")[-1]
number = parts.split("_")[1]
patient_code = "P"+number+"_"

In [31]:
# Move all subdirectories inside original folders to Patient_N folder
for folder in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder)
    for item in os.listdir(folder_path):
        source = os.path.join(folder_path, item)
        destination = os.path.join(root_path, item)
        shutil.move(source, destination)
        print(f"Done for folder {item}")

# Delete original folders
for folder in os.listdir(root_path):
    if patient_code in folder:
        shutil.rmtree(os.path.join(root_path, folder))
        print(f"Deleted: {folder}")
    elif "_Linac" in folder:
        shutil.rmtree(os.path.join(root_path, folder))
        print(f"Deleted: {folder}")
 

Done for folder 20201434_RM_1_TRATAMENTO
Done for folder 20201434_RM_1_FOLLOW_UP
Done for folder 20201434_CT_1_TRATAMENTO


In [32]:
for folder in os.listdir(root_path):
    print(folder)
    if folder.startswith("Patient_"):
        shutil.rmtree(os.path.join(root_path, folder))
        print(f"Deleted: {folder}")

20201434_RM_1_TRATAMENTO
20201434_RM_1_FOLLOW_UP
20201434_CT_1_TRATAMENTO
Patient_83
Deleted: Patient_83


### 1.2. Rename folders

In [33]:
rootPath = "/notebooks/disk2/RT_Dataset_2/01_1_Unzipped_Dataset/Patient_83" # pasta desordenada
organizePath = "/notebooks/disk2/RT_Dataset_2/01_2_Organized_Dataset/Patient_83" # pasta organizada

In [34]:
# Copy all files from the original folder to the one that will be organized
def copyFiles(rootPath, organizePath):
    if not os.path.exists(organizePath): 
        shutil.copytree(rootPath, organizePath)
        print("--> Organized directory created")
    else:
        print("--> Organized directory " + organizePath.split("/")[-2] + " already exists.")
        alltoOrganize = glob(organizePath + '*/*')
        print("--> Total of %d DICOM images." %len(alltoOrganize))

copyFiles(rootPath, organizePath)

--> Organized directory created


In [35]:
# Rename all folders from the patient to P_N
def rename_folders(organizePath):
    for folder in os.listdir(organizePath):
        to_substitute = folder.split("_",1)[0]
        to_keep = folder.split("_",1)[1]
        
        folder_path = os.path.join(organizePath, folder)
        parts = folder_path.split('/')
        patient_part = next(part for part in parts if part.startswith('Patient_'))
        patient_id = patient_part.split('_')[1]
    
        new_name = f"P{patient_id}_{to_keep}"
        new_path = os.path.join(organizePath, new_name)
        os.rename(folder_path, new_path)
        #print(f"Renamed: {folder_path} to {new_path}")
    print("--> Task completed")
        
rename_folders(organizePath)

--> Task completed


### 1.3. Simplify RM Directories

#### Para quando os ficheiros não são .dcm

In [ ]:
# Remove unnecessary subfolders and files in both follow-up and RM folders - just if the documents are not .dcm
for folder in os.listdir(organizePath):
    if "Store" in folder:
        path = os.path.join(organizePath, folder)
        os.remove(path)
        print(f"Deleted: {path}")

    elif "checkpoints" in folder:
        path = os.path.join(organizePath, folder)
        shutil.rmtree(path)
        print(f"Deleted: {path}")
        
    elif "_RM_" in folder :
        path = os.path.join(organizePath, folder)
        
        # Delete "DICOMDIR", "LOCKFILE", "VERSION" files from follow-up folder
        delete = ["DICOMDIR", "LOCKFILE", "VERSION"]
        for doc in delete:
            path_to_delete = os.path.join(path, doc)
            if os.path.exists(path_to_delete):
                os.remove(path_to_delete)
                print(f"Deleted: {path_to_delete}")

        # Get inside the first folder
        for subfolder in os.listdir(path):
            new_path = os.path.join(path, subfolder)

            # Delete"VERSION" file from the first folder 
            path_to_delete = os.path.join(new_path, "VERSION")
            if os.path.exists(path_to_delete):
                os.remove(path_to_delete)
                print(f"Deleted: {path_to_delete}")

            # Get inside the second folder
            for subsubfolder in os.listdir(new_path):
                final_path = os.path.join(new_path, subsubfolder)

                # Delete "VERSION" file from the second folder
                path_to_delete = os.path.join(final_path, "VERSION")
                if os.path.exists(path_to_delete):
                    os.remove(path_to_delete)
                    print(f"Deleted: {path_to_delete}")
                    
                if os.path.exists(final_path):
                    if os.path.isdir(final_path):
                        for item in os.listdir(final_path):
                            source = os.path.join(final_path, item)
                            destination = os.path.join(path, item)
                            shutil.move(source, destination)
                    else:
                        print(f"Error: {final_path} is not a directory.")
                else:
                    print(f"Error: {final_path} does not exist.")
                print("--> Task completed")

                # Delete the seconder folder
                shutil.rmtree(final_path)
                print(f"Deleted: {final_path}")

            # Delete the first folder
            shutil.rmtree(new_path)
            print(f"Deleted: {new_path}")
    else:
        print(f"----> No files to delete: {folder}")

#### Para quandos os ficheiros são .dcm

In [36]:
for folder in os.listdir(organizePath):
    if "Store" in folder:
        path = os.path.join(organizePath, folder)
        os.remove(path)
        print(f"Deleted: {path}")

    elif "checkpoints" in folder:
        path = os.path.join(organizePath, folder)
        shutil.rmtree(path)
        print(f"Deleted: {path}")
    else:
        print(f"----> No files to delete: {folder}")

----> No files to delete: P83_CT_1_TRATAMENTO
----> No files to delete: P83_RM_1_TRATAMENTO
----> No files to delete: P83_RM_1_FOLLOW_UP


### 1.4. Reorganize CT folders

In [37]:
# Rename plan and dose files according to the location
for folder in os.listdir(organizePath):
    print(folder)
    if "_CT_" not in folder and "_RM_" not in folder:
        nome = folder.split("_")
        print(nome)
        subfolder = os.path.join(organizePath, folder)
        for file in os.listdir(subfolder):
            actual_name = file.split("(")[0]
            actual_path = os.path.join(subfolder, file) 
            new_name = f"{actual_name}_{nome}.dcm"
            new_path = os.path.join(subfolder, new_name)
            os.rename(actual_path, new_path)
            # print(f"Renamed: {actual_path} to {new_path}")

P83_CT_1_TRATAMENTO
P83_RM_1_TRATAMENTO
P83_RM_1_FOLLOW_UP


In [38]:
# Create CT_Images folder and move all CT images to this directory
for folder in os.listdir(organizePath):
    if "_CT_" in folder or "CT_" in folder:
        folder_path = os.path.join(organizePath, folder)
        destination_folder = os.path.join(folder_path, 'CT_Images')
        
        if os.path.exists(destination_folder):
            print("CT_Images folder already exists.")
        else:
            os.makedirs(destination_folder, exist_ok=True)
            
            for filename in os.listdir(folder_path):
                #print(filename)
                if "DCMRT_Structures" not in filename and "CT_Images" not in filename and "DCMTPS_" not in filename and "DCMRT_Plan" not in filename:
                    source_file_path = os.path.join(folder_path, filename)
                    destination_file_path = os.path.join(destination_folder, filename)
                    if not os.path.exists(destination_file_path):
                        shutil.move(source_file_path, destination_file_path)
            print("--> Task completed for folder:", folder)

--> Task completed for folder: P83_CT_1_TRATAMENTO


In [39]:
# Function to copy files from one directory to another 
def copy_files_if_not_exist(source_folder, destination_folder):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    if not os.path.exists(destination_folder):
        print(f"Destination folder '{destination_folder}' does not exist.")
        return
    
    files = os.listdir(source_folder)
    
    for file in files:
        source_file_path = os.path.join(source_folder, file)
        destination_file_path = os.path.join(destination_folder, file)
        
        if not os.path.exists(destination_file_path):
            shutil.copy2(source_file_path, destination_folder)
            # print(f"Copied '{file}' to '{destination_folder}'")

In [40]:
# Add all the dose and plan files for a treatment to the same folder and delete the unnecessary folders
for number in range(1,6):
    for folder in os.listdir(organizePath):
        folder_path = os.path.join(organizePath, folder)
        parts = folder_path.split('/')
        patient_part = next(part for part in parts if part.startswith('Patient_'))
        patient_id = patient_part.split('_')[1]

        if "P"+patient_id+"_"+str(number)+"_TRATAMENTO" in folder:
            new_folder = os.path.join(organizePath, "P"+patient_id+"_CT_"+str(number)+"_TRATAMENTO")
            copy_files_if_not_exist(folder_path, new_folder)

            shutil.rmtree(folder_path)
            print(f"Deleted: {folder_path}")

## Patients with missing CT

### 1.1. Extract files from Zipped folder

In [2]:
# Extract folders from each pacient from file Paciente_N.zip
file = zipfile.ZipFile("/notebooks/disk2/RT_Dataset/01_Zipped_Directories/01_Original_Dataset/_Patient_25.zip",'r')
file.extractall("/notebooks/disk2/RT_Dataset/02_Patients_Original_Dataset/_Patient_25/")

In [3]:
unzipped_directory = "/notebooks/disk2/RT_Dataset/02_Patients_Original_Dataset/_Patient_25"

# Get the list of files and subdirectories inside the 'Patient_N' folder
folder_content = os.listdir(os.path.join(unzipped_directory, "_Patient_25"))

# Move each file/subdirectory to the main directory
for item in folder_content:
    source = os.path.join(unzipped_directory, "_Patient_25", item)
    destination = os.path.join(unzipped_directory, item)
    shutil.move(source, destination)

# Remove the '__MACOSX' directory - only for the patients uploaded from Mac
shutil.rmtree(os.path.join(unzipped_directory, "__MACOSX"))

# Remove the internal 'Patient' folder
shutil.rmtree(os.path.join(unzipped_directory, "_Patient_25"))

### 1.2. Rename folders

In [4]:
rootPath = "/notebooks/disk2/RT_Dataset/02_Patients_Original_Dataset/_Patient_25" # pasta desordenada
organizePath = "/notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25" # pasta organizada

In [5]:
# Copy all files from the original folder to the one that will be organized
def copyFiles(rootPath, organizePath):
    if not os.path.exists(organizePath): 
        shutil.copytree(rootPath, organizePath)
        print("--> Organized directory created")
    else:
        print("--> Organized directory " + organizePath.split("/")[-2] + " already exists.")
        alltoOrganize = glob(organizePath + '*/*')
        print("--> Total of %d DICOM images." %len(alltoOrganize))

copyFiles(rootPath, organizePath)

--> Organized directory created


In [6]:
# Rename all folders from the patient to P_N
def rename_folders(organizePath):
    for folder in os.listdir(organizePath):
        to_substitute = folder.split("_",1)[0]
        to_keep = folder.split("_",1)[1]
        
        folder_path = os.path.join(organizePath, folder)
        parts = folder_path.split('/')
        patient_part = next(part for part in parts if part.startswith('_Patient_'))
        patient_id = patient_part.split('_')[2]
    
        new_name = f"P{patient_id}_{to_keep}"
        new_path = os.path.join(organizePath, new_name)
        os.rename(folder_path, new_path)
        #print(f"Renamed: {folder_path} to {new_path}")
    print("--> Task completed")
        
rename_folders(organizePath)

--> Task completed


### 1.3. Simplify Follow-up Directories

In [8]:
# Remove unnecessary subfolders and files in the follow-up folder
for folder in os.listdir(organizePath):
    # Choose only follow-up folders
    if "Store" in folder:
        path = os.path.join(organizePath, folder)
        os.remove(path)
        print(f"Deleted: {path}")
    
    elif "TRATAMENTO" not in folder and "Tratamento" not in folder :
        path = os.path.join(organizePath, folder) # Caminho para a pasta do follow-up
        
        # Delete "DICOMDIR", "LOCKFILE", "VERSION" files from follow-up folder
        delete = ["DICOMDIR", "LOCKFILE", "VERSION"]
        for doc in delete:
            path_to_delete = os.path.join(path, doc)
            if os.path.exists(path_to_delete):
                os.remove(path_to_delete)
                print(f"Deleted: {path_to_delete}")

        # Get inside the first folder
        for subfolder in os.listdir(path):
            new_path = os.path.join(path, subfolder)

            # Delete "VERSION" file from this location
            path_to_delete = os.path.join(new_path, "VERSION")
            if os.path.exists(path_to_delete):
                os.remove(path_to_delete)
                print(f"Deleted: {path_to_delete}")

            # Get inside the second folder
            for subsubfolder in os.listdir(new_path):
                final_path = os.path.join(new_path, subsubfolder)

                # Delete "VERSION" file from this location
                path_to_delete = os.path.join(final_path, "VERSION")
                if os.path.exists(path_to_delete):
                    os.remove(path_to_delete)
                    print(f"Deleted: {path_to_delete}")

                # Move the files to the main folder
                for item in os.listdir(final_path):
                    source = os.path.join(final_path, item)
                    destination = os.path.join(path, item)
                    shutil.move(source, destination)
                print("--> Task completed")

                # Delete the second folder
                shutil.rmtree(final_path)
                print(f"Deleted: {final_path}")

            # Delete the first folder
            shutil.rmtree(new_path)
            print(f"Deleted: {new_path}")
    else:
        print("----> No files to delete")

----> No files to delete
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/DICOMDIR
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/LOCKFILE
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/VERSION
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/YTPIUPS3/VERSION
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/YTPIUPS3/E0ZVYHTG/VERSION
--> Task completed
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/YTPIUPS3/E0ZVYHTG
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_14_06_19/YTPIUPS3
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_03_12_19/DICOMDIR
Deleted: /notebooks/disk2/RT_Dataset/03_Patients_Organized_Dataset/_Patient_25/P25_RM_03_12

### 1.4. Reorganize MR folders

In [9]:
# Create MR_Images folder and move all MR images to this directory. Then delete the unnecessary files
for folder in os.listdir(organizePath):
    if "_RM_" in folder and "TRATAMENTO" in folder:
        folder_path = os.path.join(organizePath, folder)
        destination_folder = os.path.join(folder_path, 'MR_Images')
        
        if os.path.exists(destination_folder):
            print("MR_Images folder already exists.")
        else:
            os.makedirs(destination_folder, exist_ok=True)

            exclude_files = ['RTSS.dcm', 'RTPLAN.dcm', 'RTDOSE.dcm', 'MR_Images']

            for filename in os.listdir(folder_path):
                if filename not in exclude_files:
                    source_file_path = os.path.join(folder_path, filename)
                    destination_file_path = os.path.join(destination_folder, filename)
                    if not os.path.exists(destination_file_path):
                        shutil.move(source_file_path, destination_file_path)
            print("--> Task completed for folder:", folder)

--> Task completed for folder: P25_RM_1_TRATAMENTO
--> Task completed for folder: P25_RM_2_TRATAMENTO


### 1.4. Reorganize CT folders

In [10]:
# Create CT_Images folder and move all CT images to this directory. Then delete the unnecessary files
for folder in os.listdir(organizePath):
    if "_CT_" in folder:
        folder_path = os.path.join(organizePath, folder)
        destination_folder = os.path.join(folder_path, 'CT_Images')
        
        if os.path.exists(destination_folder):
            print("CT_Images folder already exists.")
        else:
            os.makedirs(destination_folder, exist_ok=True)

            exclude_files = ['RTSS.dcm', 'RTPLAN.dcm', 'RTDOSE.dcm', 'CT_Images']

            for filename in os.listdir(folder_path):
                if filename not in exclude_files:
                    source_file_path = os.path.join(folder_path, filename)
                    destination_file_path = os.path.join(destination_folder, filename)
                    if not os.path.exists(destination_file_path):
                        shutil.move(source_file_path, destination_file_path)
            print("--> Task completed for folder:", folder)


--> Task completed for folder: P25_CT_1_TRATAMENTO
